In [121]:
## This notebook compares Palliative Care spending (2024) and Total Medicare spending
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

df = pd.read_csv('Foundry Data.csv')

In [144]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# Prepare data
# Dependent variable (no log transformation)
Y = df['TOT_MDCR_STDZD_PYMT_PC']

# Independent variables (no log transformation)
columns = ['HOSPC_MDCR_STDZD_PYMT_PCT', 'AMBLNC_MDCR_STDZD_PYMT_PCT', 'TRTMNTS_MDCR_PYMT_PCT', 
           'ASC_MDCR_STDZD_PYMT_PCT', 'TESTS_MDCR_STDZD_PYMT_PCT', 'FQHC_RHC_MDCR_STDZD_PYMT_PCT']
X = df[columns]

# Include other variables (non-transformed)
X.loc[:, ['EstimateHouseholdsMedian_income_dollars', 'BENES_WTH_PTAPTB_CNT', 
   'SNF_CVRD_STAYS_PER_1000_BENES', 'HH_EPISODES_PER_1000_BENES', 'BENE_AVG_RISK_SCRE', 'mode_medicare_pricing_for_new_patient']] = \
df[['EstimateHouseholdsMedian_income_dollars', 'BENES_WTH_PTAPTB_CNT', 
     'SNF_CVRD_STAYS_PER_1000_BENES', 'HH_EPISODES_PER_1000_BENES', 'BENE_AVG_RISK_SCRE', 'mode_medicare_pricing_for_new_patient']]

# Add STATE fixed effects
# Handle missing values in 'STATE' explicitly
if df['STATE'].isnull().any():
    df['STATE'] = df['STATE'].fillna('UNKNOWN')

state_dummies = pd.get_dummies(df['STATE'], prefix="STATE", drop_first=True).astype(int)
X = pd.concat([X, state_dummies], axis=1)

# Remove states with low frequency
low_freq_states = state_dummies.sum(axis=0)[state_dummies.sum(axis=0) < 10].index
X = X.drop(columns=low_freq_states)

# Ensure numeric and handle missing values
X = X.apply(pd.to_numeric, errors='coerce').replace([np.inf, -np.inf], np.nan)
Y = pd.to_numeric(Y, errors='coerce')

# Drop rows with missing values
combined = pd.concat([X, Y], axis=1).dropna()
X = combined[X.columns]
Y = combined[Y.name]

# Scale numeric predictors (excluding state dummies)
numeric_columns = X.select_dtypes(include=[np.number]).columns.difference(state_dummies.columns)
scaler = StandardScaler()
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])

# Add constant
X = sm.add_constant(X)

# Check VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

# Fit the OLS model
model = sm.OLS(Y, X).fit(cov_type='HC3')
print(model.summary())

# Save model summary
with open('model_summary.txt', 'w') as f:
    f.write(model.summary().as_text())


/var/folders/3x/_13b6_kn2y546q3tx2zbrv180000gn/T/ipykernel_11324/3306264316.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_columns] = scaler.fit_transform(X[numeric_columns])


                                    feature         VIF
0                                     const  213.397435
1                 HOSPC_MDCR_STDZD_PYMT_PCT    1.679473
2                AMBLNC_MDCR_STDZD_PYMT_PCT    1.534016
3                     TRTMNTS_MDCR_PYMT_PCT    1.767975
4                   ASC_MDCR_STDZD_PYMT_PCT    1.831217
5                 TESTS_MDCR_STDZD_PYMT_PCT    2.783929
6              FQHC_RHC_MDCR_STDZD_PYMT_PCT    1.717246
7   EstimateHouseholdsMedian_income_dollars    1.425387
8                      BENES_WTH_PTAPTB_CNT    1.473655
9             SNF_CVRD_STAYS_PER_1000_BENES    1.968841
10               HH_EPISODES_PER_1000_BENES    2.726134
11                       BENE_AVG_RISK_SCRE    2.190604
12    mode_medicare_pricing_for_new_patient    4.347565
13                                 STATE_AL    6.623357
14                                 STATE_AR    7.658543
15                                 STATE_AZ    2.116531
16                                 STATE_CA    4

In [147]:
import pandas as pd
import statsmodels.api as sm

def create_regression_table_unscaled(model, scaler, pct_vars, output_file="regression_table.csv"):
    """
    Create a regression table with unscaled coefficients (and confidence intervals) 
    for the variables listed in pct_vars. This assumes the dependent variable was not scaled,
    but these pct_vars were standardized using the same 'scaler'.
    """
    # Copy model results
    unscaled_params = model.params.copy()      # scaled coefficients
    unscaled_se = model.bse.copy()            # scaled standard errors
    unscaled_ci = model.conf_int()            # scaled confidence intervals

    # For each PCT predictor that was scaled, divide by its standard deviation from 'scaler'
    for var in pct_vars:
        if var in unscaled_params.index:
            var_idx = list(scaler.feature_names_in_).index(var)
            scale_val = scaler.scale_[var_idx]

            unscaled_params[var] /= scale_val
            unscaled_se[var] /= scale_val

            unscaled_ci.loc[var, 0] /= scale_val
            unscaled_ci.loc[var, 1] /= scale_val

    # Build output table using the UNscaled params & SE but keep original z-stats and p-values
    results = {
        "Variable": unscaled_params.index,
        "Coefficient": unscaled_params.values,
        "Std. Error": unscaled_se.values,
        "z-stat": model.tvalues.values,
        "P>|z|": model.pvalues.values,
        "95% CI": [
            f"[{low:.3f}, {high:.3f}]"
            for low, high in zip(unscaled_ci[0], unscaled_ci[1])
        ]
    }

    results_df = pd.DataFrame(results)

    # Format numeric columns
    results_df["Coefficient"] = results_df["Coefficient"].apply(lambda x: f"{x:.3f}")
    results_df["Std. Error"] = results_df["Std. Error"].apply(lambda x: f"{x:.3f}")
    results_df["z-stat"] = results_df["z-stat"].apply(lambda x: f"{x:.3f}")
    results_df["P>|z|"] = results_df["P>|z|"].apply(lambda x: f"{x:.3f}")

    # Save to CSV
    results_df.to_csv(output_file, index=False)
    print(f"Unscaled regression table saved to {output_file}")


# Example usage (assume `model` is your fitted OLS regression model, and `scaler` is the trained StandardScaler):
pct_vars = [
    'HOSPC_MDCR_STDZD_PYMT_PCT',
    'AMBLNC_MDCR_STDZD_PYMT_PCT',
    'TRTMNTS_MDCR_PYMT_PCT',
    'ASC_MDCR_STDZD_PYMT_PCT',
    'TESTS_MDCR_STDZD_PYMT_PCT',
    'FQHC_RHC_MDCR_STDZD_PYMT_PCT'
]

create_regression_table_unscaled(model, scaler, pct_vars, output_file="regression_table.csv")


Unscaled regression table saved to regression_table.csv
